In [2]:
#1. Load base model - the same used for training
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import bitsandbytes
from dotenv import load_dotenv
import os
import dvc.api
load_dotenv()


True

In [1]:
# Base model
base_model = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(base_model)
hf_token = os.getenv("HF_TOKEN")


# Load base model in 4bit (same as training)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    device_map="auto",
    token=hf_token
)


The 8-bit optimizer is not available on your device, only available on CUDA for now.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
D:\DEV\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 2471.65 MB. The target location C:\Users\mk666\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B\blobs only has 2185.76 MB free disk space.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

D:\DEV\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mk666\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [11]:
# Lets now load model weights from dvc

path_adapter_config = 'trained_model/adapter_config.json'
path_adapter_model = 'trained_model/adapter_model.safetensors'
repo = '.'
version = 'llama3.2-1b_model_adapter_v1' # Always remember the tags!

adapter_config_url = dvc.api.get_url(
	path=path_adapter_config,
	repo=repo,
	rev=version
	)

adapter_model_url = dvc.api.get_url(
	path=path_adapter_model,
	repo=repo,
	rev=version
	)

print(adapter_config_url)
print(adapter_model_url)

s3://my-dvc-shared-bucket-123/dvc_storage/files/md5/e4/74bf1fb6ae75a7932f6c1a990e6694
s3://my-dvc-shared-bucket-123/dvc_storage/files/md5/f7/3b89333b11fea518d9d599434ea270


In [12]:
# Load LoRA adapters (from your trained_model dir tracked with DVC)
model = PeftModel.from_pretrained(model, "./trained_model")

In [26]:
def chat(query, model, tokenizer):
    system_prompt = """"""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    
    # Use same template as training
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to("cpu")

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [27]:
tokenizer.chat_template = (
    "{% set loop_messages = messages %}"
    "{% for message in loop_messages %}"
    "{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n' + message['content'] | trim + '<|eot_id|>' %}"
    "{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}"
    "{{ content }}"
    "{% endfor %}"
    "{% if add_generation_prompt %}{{ '<|start_header_id|><|end_header_id|>\n\n' }}{% endif %}"
)

In [2]:
# Example usage
print(chat("I want a domain for a startup that helps people sleep better", model, tokenizer))

NameError: name 'chat' is not defined